In [1]:
import sys
if "../" not in sys.path: sys.path.insert(0,"../");

In [7]:
from typing import Tuple
from experimental.sed import MorphologyAnalyzer
analyzer = MorphologyAnalyzer("../resources/pretrained/models_sed_morpheme_template.txt")

def break_word(word: str, analyzer) -> Tuple[str]:
    return tuple([ su for su in analyzer.break_text([word])[word]])


In [13]:
from typing import Iterable, List
from marynlp import funcutils as f

from tqdm import tqdm

@f.apply(set)
def get_unique_item_from_item_sequences(word_sequences: Iterable[Iterable[str]]):
    for word_seq in word_sequences:
        for word in word_seq:
            word = word.strip()
            if len(word) != 0:
                yield word

@f.apply(set)
def get_unique_morphemes(words: Iterable[str], analyzer_break_word):
    for word in tqdm(words):
        for morph in analyzer_break_word(word):
            yield morph


def read_text_from_files(txt_file_path: str):
    with open(txt_file_path, mode="r") as rb:
        return rb.readlines()
    
@f.apply(set)
def load_tokens(store_path):
    with open(str(store_path), 'r') as file:
        for y in json.load(file):
            for i in y:
                yield i

import re
read_word_sequence_from_file = f.forEach(lambda l: re.split(r"\s+", l.strip()))(read_text_from_files)


In [20]:
from marynlp import funcutils as f

import json

from marynlp.text.data import Vocab
from marynlp.text.processor import TokenEncoder
from marynlp.text.processor import BaseTokenEncoder 

word_vocab = Vocab(unique_words)

word_tokenizer = BaseTokenEncoder(word_vocab.get_tokens() + ['<UNK>'])
morpheme_tokenizer = TokenEncoder(Vocab(load_tokens("../resources/pretrained/models_small-tkns.txt")))

In [21]:
from experimental.sed.modules import CompositionalEmbeddings, CompositionalLanguageModel

morph_embedder = CompositionalEmbeddings(
    morpheme_vocab_size=morpheme_tokenizer.size,
    embedding_dim = 512,
    hidden_dim = 512,
    composition_fn = 'rnn',
    use_cuda=False
);

In [22]:
from marynlp.utils.storage import download as dl
from marynlp.utils import storage
from marynlp.utils.storage import local

sed_small_embeddings_pth = local.get_path_from_store("./sed_small_embeddings.pth")

if not sed_small_embeddings_pth.exists():
    print("Downloading %s..." % (sed_small_embeddings_pth))
    # create folders
    sed_small_embeddings_pth.parent.mkdir(exist_ok=True, parents=True)
    sed_small_embeddings_pth = dl.file_from_google("models/embeddings/small-embeddings.pth", bucket, str(sed_small_embeddings_pth))

morph_embedder.update_weight_from_path(sed_small_embeddings_pth)